The old one: We only have so much gradient. Some of the gradient goes the alpha because it's one of 4 things it can optimize. So, because the alpha is helping to optimize the image where it's important, it clears out that alpha region first. Then this leads to a vicious cycle where the blocked areas aren't optimized and therefore its not safe to uncover them. this happens until the below image becomes palatable by chance.

Todo: two way occlusion. Then, things like "stuck in the bathtub" can occlude a part of the foreground as well. OR we could solve this by having two layers...sandwiching the foreground...

In [ ]:
import rp

import nerf.sd as sd
import torch
import torch.nn as nn
from ryan.source.learnable_textures import LearnableTexturePackRaster,LearnableTexturePackFourier
from ryan.source.learnable_textures import LearnableImageRaster,LearnableImageFourier
import icecream
import itertools
from IPython.display import clear_output
import textwrap

In [ ]:
if 's' not in dir():
    s=sd.StableDiffusion('cuda')
    # s=sd.StableDiffusion('cuda',"nitrosocke/Arcane-Diffusion")
    # s=sd.StableDiffusion('cuda',"/home/ryan/CleanCode/Datasets/models/stable_diffusion/v1_4/dreambooths/dreambooth-kurokiyousei-2000step-1666174076529")
device=s.device

In [ ]:
prompts=[
    # "beautiful sks jinx, Greg Rutkowski, ilya kuvshinov,artgerm highly detailed, flat stomach, hourglass figure"
    "arcane style, jinx"
    # "kurokiyousei"
]*5
for prompt in prompts:
    print(prompt)
    img=s.prompt_to_img(prompt)[0]
    rp.display_image(img)

In [ ]:
img[0]

In [ ]:
height=width=128

def make_learnable_image(num_channels):
    return LearnableImageFourier(height,width,num_channels,num_features=512)
    
def composite_images(bottom,top,alpha):
    return alpha*top+(1-alpha)*bottom

class LearnableImageRGBA(nn.Module):
    def __init__(self):
        super().__init__()
        self.image=make_learnable_image(4)
        
    def forward(self):
        image=self.image()
        assert image.shape==(4,height,width)
        rgb=image[:3]
        alpha=image[3:]
        assert rgb.shape  ==(3,height,width)
        assert alpha.shape==(1,height,width)
        alpha=alpha.repeat(3,1,1)
        assert alpha.shape==rgb.shape==(3,height,width)
        return rgb,alpha
    
class LearnableImageRGB(nn.Module):
    def __init__(self):
        super().__init__()
        self.image=make_learnable_image(3)
        
    def forward(self):
        rgb=self.image()
        assert rgb.shape  ==(3,height,width)
        return rgb

In [ ]:
#Cartridge: First attempt!
prompt_sections = [
    [
        # # 'danny devito',
        # 'mario',
        # 'a cute puppy',
        'arcane style, a violin',
        # 'an amethyst crystal',
        # # 'pikachu',
        # # 'a rubiks cube',
        # 'a spooky skeleton',
        # 'hatsune miku',
        # # 'a tesla car',
        # 'a steamboat',
        'arcane style, a mermaid',
        # 'a photo of a bananna',
        # 'an astronaut riding a horse',
        # 'a lolita outfit',
        # # 'a cactus',
        # # 'a campfire',
        # # 'a teddy bear',
        'arcane style, a bulldozer',
        'arcane style, a nuclear blast mushroom cloud',
        # 'a cute anime girl',
        'arcane style, 3d emoji of sans from undertale',
        # 'emma watson',
        # 'emma watson',
        # 'emma watson',
        # 'emma watson',
        # 'emma watson',
        # 'emma watson',
        # 'emma watson',
        # 'emma watson',
        # 'emma watson',
        # 'emma watson',
        'arcane style, shrek shrek shrek',
        'arcane style, danny devito',
        'arcane style, hatsune miku',
        'arcane style, a 3d anime girl',
        'arcane style, a 3d anime boy',
        'arcane style, a mushroom man',
        'arcane style, spaghetti and meatballs',
        'arcane style, a 3d pixar woman',
        'arcane style, a 3d pixar man',
        'arcane style, jean luc picard',
        # 'arcane style, emma watson',
        'arcane style, sonic the hedgehog',
        'arcane style, darth vader',
        'arcane style, a cute dog',
        'arcane style, a teddy bear',
        'arcane style, a cat',
        'arcane style, two spooky skeletons kissing',
        'arcane style, emma watson holding a puppy',
        'arcane style, jinx',
    ],
    [
        # # '',
        # # # 'floating in deep space',
        'in the backrooms',
        'in outer space',
        'in the living room',
        # # # 'in a suit of armor',
        'on the front lawn',
        'stuck in the bathtub',
        'in a wooden wheelbarrow',
        'in an old stone castle',
        'in a fish tank',
        'in times square',
        'burning in hell',
        'in a flying saucer',
        'in a corn field',
        'on a boat',
        'in the desert',
        'on mars',
        'in a deep cave',
        # 'riding a horse',
        
#         'wearing a tin foil hat',
#         'wearing a tophat',
#         'wearing a baseball cap',
#         'wearing a witch hat',
#         'wearing a biking helmet',
#         'wearing a bandana',
        # 'wearing a blonde wig',
        
        # 'eating a burger',
        # 'eating a hogie',
        # 'eating a slice of pizza',
        # 'drinking a glass of water',
        # 'drinking from a water bottle',
        # 'drinking a glass of wine',
        # 'eating a lollipop',
        # 'eating a bananna',
        # 'eating a pretzel',
        
        # # 'wearing a white roman toga',
        # 'wearing a lolita dress',
        # 'wearing a blue t-shirt',
        # 'wearing a striped wool sweater',
        # 'wearing a chrome suit of armor',
        # 'wearing a button down t-shirt',
        # # 'wearing a yellow hazmat suit',
        # 'in a miniskirt',
        # 'covered in spiders',
        # 'in blue overalls',
        # 'wearing a blonde wig',
        # 'wearing a pointy hat',
        # 'in a bath robe',
        # # 'holding a sword',/
        # # 'holding a frying pan',
        # # 'eating cheese',
    ],
]

REVERSE=True #Swap what is background for what is foreground without altering the prompts.
REVERSE=False
prompt_structure = '%s %s'

if not REVERSE:

    prompts = [prompt_structure%sections for sections in itertools.product(*prompt_sections)]
    
else:
    prompt_sections=prompt_sections[::-1]
    prompts = [prompt_structure%sections[::-1] for sections in itertools.product(*prompt_sections)]

foreground_prompts, background_prompts = prompt_sections
TRAIN_BACK=False #Should we train the backgrounds alone? (Yes leads to better performance - no is for asking interesting ablation questions)
TRAIN_BACK=True 

old_prompts=list(prompts)
# prompts += foreground_prompts
prompts += background_prompts
num_singles = len(prompts)-len(old_prompts)


icecream.ic(prompts);
text_embeddings=[s.get_text_embeddings(prompt) for prompt in prompts]




images_foregrounds=[LearnableImageRGBA().to(device) for _ in prompt_sections[0]]
images_backgrounds=[LearnableImageRGB ().to(device) for _ in prompt_sections[1]]

all_images=images_foregrounds+images_backgrounds

In [ ]:
def get_images(include_fb=False):
    foregrounds=[x() for x in images_foregrounds]
    backgrounds=[x() for x in images_backgrounds]
    
    composites=[composite_images(f,b,ɑ) for (f,ɑ),b in itertools.product(foregrounds,backgrounds)]
    
    composites += backgrounds
    # composites += [x[0] for x in foregrounds]
        
    if include_fb:
        return composites,foregrounds,backgrounds
    else:
        return composites

In [ ]:
dimgs=[]

In [ ]:
def do_display(iter_num=None):
    
    with torch.no_grad():
        
        date_string=rp.r._format_datetime(rp.get_current_date())[4:]#Get rid of 'Tue ' or 'Wed ' etc
        stats_string="""gradient_mixer.ipynb\n%s\niter_num: %i \nLEARNING_RATE: %.1E\nALPHA_HATE: %.1E \nTRAIN_BACK: %s \nREVERSE: %s\nBATCH_SIZE: %i""".strip()%\
            (date_string, iter_num, LEARNING_RATE, ALPHA_HATE, TRAIN_BACK, REVERSE, BATCH_SIZE)
        stats_string=rp.cv_text_to_image(stats_string,scale=.5)
        stats_string = rp.cv_resize_image(stats_string,2/3)
        
        imgs,f,b=get_images(include_fb=True)
        f,ɑ=zip(*f)
        imgs=rp.as_numpy_images(torch.stack(imgs))
        f=rp.as_numpy_images(torch.stack(f))
        ɑ=1-rp.as_numpy_images(torch.stack(ɑ))
        b=rp.as_numpy_images(torch.stack(b))
        
        
        #Add borders to each image
        def borderize(img):
            return rp.bordered_image_solid_color(img,color=(0,0,0,0),thickness=0,bottom=1,right=1)
        f=list(map(borderize,f))
        ɑ=list(map(borderize,ɑ))
        b=list(map(borderize,b))
        imgs=list(map(borderize,imgs))
        
        
        def labeled_images(images,labels,position='top',just_labels=False):
            if just_labels:
                images=[image[0:1,:]*0 for image in images]#Make them all lines...we just want the labels
            colors=[(200,250,250),(250,250,200),(250,200,250)]
            text_width=10
            wrapped_labels=[rp.line_join(textwrap.wrap(label,text_width)) for label in labels]
            num_lines=max(rp.number_of_lines(x) for x in wrapped_labels)
            wrapped_labels=['\n'*(num_lines-rp.number_of_lines(x))+x for x in wrapped_labels]
            return [rp.labeled_image(images[i],
                                     wrapped_labels[i],
                                     size=15*rp.number_of_lines(wrapped_labels[i]),
                                     position=position,
                                     text_color=colors[i%len(colors)])
                    for i in range(len(labels))],num_lines
        
        # top_dimg=rp.tiled_images(rp.list_pop(list(zip(*rp.split_into_sublists(imgs[:-num_singles],len(images_backgrounds))))),length=len(images_foregrounds))
        # top_dimg=rp.tiled_images(rp.list_pop(list(zip(*rp.split_into_sublists(imgs[:-num_singles],len(images_backgrounds))))),length=len(images_foregrounds))
        imggrid=list(zip(*rp.split_into_sublists(imgs[:-num_singles],len(images_backgrounds))))
        imggrid=list(map(list,imggrid))
        singles=imgs[-num_singles:]
        singles,num_bg_lines=labeled_images(singles,background_prompts,position='left')
        for i,single in enumerate(singles):
            imggrid[i].insert(0,single)


        # top_dimg=rp.tiled_images(imgs[:-num_singles],length=len(images_backgrounds))
        # bot_dimg=rp.tiled_images(imgs[-num_singles:])
        blank_image=rp.as_numpy_array([[0.0]])
        blank_image=rp.uniform_float_color_image(height,width,color=(0,0,0,0))
        ɑ_labels,_=labeled_images(ɑ,foreground_prompts,just_labels=True)

        imggrid.insert(0,[
            rp.labeled_image(
                rp.labeled_image( blank_image[15:], 'Back  ' ,text_color=(255,128 if TRAIN_BACK else 0,0),position='bottom', size=15,align='right'),
                '\n'*(num_bg_lines-1)+'Front'  ,size=15*num_bg_lines,text_color=(255,0,0),position='right',flip_text=True)
        ]+list(f))
        imggrid.insert(0,[rp.labeled_image(blank_image,'\n'*(num_bg_lines-1)+'Alpha',size=15*num_bg_lines,text_color=(255,128 if ALPHA_HATE else 0,0),position='right',flip_text=True)]+list(ɑ))
        imggrid.insert(0,[blank_image[0:1,0:1]]+list(ɑ_labels))
        # mggrid.insert(0,[blank_image+list(ɑ))
        dimg=rp.grid_concatenated_images(imggrid)
        
        # extras=rp.horizontally_concatenated_images(ɑ)
        # dimg=rp.vertically_concatenated_images(extras,dimg)
        dimg=rp.blend_images(dimg,stats_string,stats_string*1/2)
        
        dimgs.append(dimg)
        rp.display_image(dimg)

In [ ]:
dimgs.clear()

In [ ]:
iter_num=0

In [ ]:
params=list(itertools.chain(*(x.parameters() for x in all_images)))
icecream.ic(len(params),TRAIN_BACK,num_singles)

LEARNING_RATE=1e-3/5

optim=torch.optim.Adam(params,lr=1e-3/5)#For fourier. 1e-2 is too high.
# optim=torch.optim.Adam(params,lr=1e-2)#Normally this is too high, as the colors start to get too strong...but in this case, we want a binary mask anyway lol - might as well do it the fast way!
# optim=torch.optim.Adam(params,lr=1e-2/5)#Normally this is too high, as the colors start to get too strong...but in this case, we want a binary mask anyway lol - might as well do it the fast way!
# optim=torch.optim.Adam(params,lr=1e-1)#For raster. 
# optim=torch.optim.SGD(params,lr=1e-1)#For raster
# optim=torch.optim.SGD(params,lr=1e-5)#For raster

ALPHA_HATE=1e-2 #If this is true, we hate alpha! We want it to be transparent
# ALPHA_HATE=1e-1 #If this is true, we hate alpha! We want it to be transparent
ALPHA_HATE=False

BATCH_SIZE=5
BATCH_SIZE=10

display_eta=rp.eta(1000000)

for _ in range(1000000):
    __=_
    _=iter_num
    iter_num+=1
    display_eta(_)
    
    
    imgs,f,b=get_images(include_fb=True)
    if not TRAIN_BACK:
        imgs=imgs[:-num_singles]
    
    f,a=zip(*f)
    batch=list(zip(text_embeddings,imgs))
    
    if ALPHA_HATE:
        (-torch.stack(a).sum()*ALPHA_HATE).backward(retain_graph=True) #Alpha pushback loss. unbalanced;varies with num imgs
    
    # batch_size=5
    # batch_size=5 #I'm going to sleep...I'll let this run overnight. This batch size is overkill tho
    
    mini_batch = rp.random_batch(batch,min(BATCH_SIZE,len(batch)))
    
    for embed,image in mini_batch:
        s.train_step(embed,image[None],by_loss=False)
        
    
    with torch.no_grad():
        if _ and not _%500:
            clear_output()
            # rp.display_image_slideshow(dimgs)
        if not _%50 or not __:
            do_display(iter_num)

    optim.step()
    optim.zero_grad()

In [ ]:
rp.display_image(dimgs[-1])

In [ ]:
rp.display_image_slideshow(dimgs,)

In [ ]:
rp.display_image_slideshow(dimgs[::20])

In [ ]:
do_display()

In [ ]:
rp.display_image_slideshow(dimgs)

In [ ]:
do_display()

In [ ]:
rp.display_image_slideshow(dimgs)

In [ ]:
rp.display_image_slideshow(dimgs)

In [ ]:
rp.display_image_slideshow(dimgs)

In [ ]:
rp.display_image_slideshow(dimgs)

In [ ]:
rp.display_image_slideshow(dimgs)

In [ ]:
rp.display_image_slideshow(dimgs)

In [ ]:
rp.display_image_slideshow(dimgs)

In [ ]:
rp.display_image_slideshow(dimgs)

In [ ]:
rp.display_image_slideshow(dimgs)